اول حاجه هنحط الداتا في مونجو دي بي 

In [ ]:
#this is new DAG
import json
from pymongo import MongoClient

# 1️⃣ الاتصال بـ MongoDB
client = MongoClient("mongodb://localhost:27017/")

# 2️⃣ اختيار Database و Collection
db = client["nyc_data"]
collection = db["nyc_traffic"]

# (اختياري) تفريغ الكوليكشن لو بتجرب
# collection.delete_many({})

# 3️⃣ قراءة ملف الـ JSON
with open("nyc_traffic.json", "r") as f:
    data = json.load(f)

# تأكيد إن الداتا Array
assert isinstance(data, list), "JSON file must contain a list of documents"

# 4️⃣ إدخال الداتا
collection.insert_many(data)

print(f"✅ Inserted {len(data)} documents into MongoDB successfully")


✅ Inserted 41362 documents into MongoDB successfully


حولنا الداتا بتتاعت الشوارع من ملف سي اي في اي  ملف  parquet
ودي عشان ننوع من مصارد الداتا الي جايلنا 

In [1]:

import pandas as pd 
data = pd.read_csv("Centerline.csv")
data.to_parquet("streets_data.parquet")

انشاء اول تاسك في الداج وده هنعمل كونكت مع الاس 3 و هنكريت باكت 
ونعمل فيها 2 فولدر وبعدين نرفع الرو داتا فيها 
1- هنجيب الداتا  من مونجو دي بي ونرفعها علي هئيه جيسون فايل 
2- هنجيب الفايل البركيه ونرفعه 
3- هنجيب الفايل الثالث والاخير ونرفعه الي هو csv 

In [12]:
from pymongo import MongoClient
import json
import boto3

# اتصال Mongo
client = MongoClient("mongodb://localhost:27017")
db = client["nyc_data"]
collection = db["nyc_traffic"]

#-----------------------------------------------
# connect with s3 
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)
Bucket="datalake"

#------------------------------------------------
# create bucket لو مش موجود 
try:
     s3.create_bucket(Bucket="datalake")
except:
     pass

#------------------------------------------------
# create 2 folder (raw , processed)
try:
     s3.put_object(Bucket="datalake", Key="raw/")
     s3.put_object(Bucket="datalake", Key="processed/")
except:
     pass

#----------------------------------------------------------
# upload first file to s3 (json file)
cursor = collection.find({}, {"_id": 0})

data = list(cursor)          # in-memory
json_bytes = json.dumps(
    data,
    ensure_ascii=False
).encode("utf-8")

#load to s3
s3.Bucket("datalake").put_object(
    Key="raw/mongo_traffic.json",
    Body=json_bytes,
    ContentType="application/json"
)


#---------------------------------------------------==
# upload second file csv to s3
s3_client = boto3.client(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)


s3_client.upload_file(
    Filename="Motor_Vehicle_Collisions_Crashes.csv",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/Motor_Vehicle_Collisions_Crashes.csv",    # المسار داخل الباكيت
    ExtraArgs={"ContentType": "text/csv"}   # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)

#--------------------------------------
# upload the last file (parquet file )

s3_client.upload_file(
    Filename="streets_data.parquet",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/streets_data.parquet" ,   # المسار داخل الباكيت
    ExtraArgs={"ContentType": "application/octet-stream"}  # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)




التاسك التاني هنجيب الداتا بتاعت الجيسون فايل ونحولها لداتا فريم 
دا بستخدام باي سبارك و نعمل عليها بروسيسنج وبعدين نرفعها علي س3 تاني بس المره ده في فولدر اسمه بروسيس داتا 


In [ ]:
import boto3
import pandas as pd
from io import BytesIO
from pyspark.sql import SparkSession

# -------------------------
# 1️⃣ إعداد MinIO/S3
# -------------------------
s3 = boto3.client(
    "s3",
    endpoint_url="http://localhost:9010",  # غيرها لو عندك S3 حقيقي
    aws_access_key_id="minioadmin",
    aws_secret_access_key="minioadmin123"
)

# -------------------------
# 2️⃣ قراءة json من S3
# -------------------------
obj = s3.get_object(Bucket="datalake", Key="raw/mongo_traffic.json")
pdf = pd.read_csv(BytesIO(obj['Body'].read()))

# -------------------------
# 3️⃣ تحويل Pandas لـ PySpark DataFrame
# -------------------------
spark = (SparkSession.builder
        .appName("transform 1")
        .getOrCreate())
df = spark.createDataFrame(pdf)

# -------------------------
# 4️⃣ أي Processing محتاجينه
# -------------------------
# مثال: خلي العمود 'CRASH_DATE' كـ تاريخ
# from pyspark.sql.functions import col, to_date
# df = df.withColumn("CRASH_DATE", to_date(col("CRASH_DATE"), "MM/dd/yyyy"))

df.show(5)

# -------------------------
# 5️⃣ تحويل PySpark DataFrame لـ Pandas
# -------------------------
#pdf_processed = df.toPandas()

# -------------------------
# 6️⃣ رفع CSV جديد على S3/MinIO باسم processed/
# -------------------------
# csv_buffer = BytesIO()
# pdf_processed.to_csv(csv_buffer, index=False)

# s3.put_object(
#     Bucket="datalake",
#     Key="processed/mongo_traffic.json",
#     Body=csv_buffer.getvalue()
# )

